In [10]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/a8/dc/e5797d6bf966cd3baa5a6ae32bec31472934c9021ca1505dc7bf5c8fc902/tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-pasta>=0.1.1 from https://files.pythonhosted.org/packages/a3/de/c648ef6835192e6e2cc03f40b19eeda4382c49b5bafb43d88b931c4c74ac/google_pasta-0.2.0-py3-none-any.whl.metadata
  Obtaining dependen

In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, random_split
import tensorflow as tf
import os
import cv2
from PIL import Image
import seaborn as sns
import time
import datetime

2024-04-17 13:47:20.397188: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 13:47:20.443211: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 13:47:22.395614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
import shutil

In [3]:
#!find Train -name .ipynb_checkpoints -exec rm -r {} +

In [2]:
enable_wandb = True
use_gpu = True
gpu_available = torch.cuda.is_available()
gpu_available

True

In [3]:
if enable_wandb:
  !pip install wandb -qU
  import wandb
  wandb.login()


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: evafan12 (evafan123). Use `wandb login --relogin` to force relogin


### Data

In [34]:
import os

def count_images_per_category(directory):
    category_image_count = {}

    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        
        if os.path.isdir(category_path):
            num_images = len([item for item in os.listdir(category_path) if item.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])
            category_image_count[category] = num_images

    return category_image_count

dataset_directory = 'Train'
image_counts = count_images_per_category(dataset_directory)
sorted_train = {key: image_counts[key] for key in sorted(image_counts.keys(), key=lambda x: int(x))}

print(sorted_train)
total_train = sum(sorted_train.values())
print(f"Total images across all categories for training: {total_train}")

{'00': 210, '01': 2220, '02': 2250, '03': 1410, '04': 1980, '05': 1860, '06': 420, '07': 1440, '08': 1410, '09': 1470, '10': 2010, '11': 1320, '12': 2100, '13': 2160, '14': 780, '15': 630, '16': 420, '17': 1110, '18': 1200, '19': 210, '20': 360, '21': 330, '22': 390, '23': 510, '24': 270, '25': 1500, '26': 600, '27': 240, '28': 540, '29': 270, '30': 450, '31': 780, '32': 240, '33': 689, '34': 420, '35': 1200, '36': 390, '37': 210, '38': 2070, '39': 300, '40': 360, '41': 240, '42': 240}
Total images across all categories for training: 39209


In [35]:
dataset_directory = 'Test_organized'
image_counts = count_images_per_category(dataset_directory)
sorted_test = {key: image_counts[key] for key in sorted(image_counts.keys(), key=lambda x: int(x))}

print(sorted_test)
total_test = sum(sorted_test.values())
print(f"Total images across all categories for testing: {total_test}")

{'0': 60, '1': 720, '2': 750, '3': 450, '4': 660, '5': 630, '6': 150, '7': 450, '8': 450, '9': 480, '10': 660, '11': 420, '12': 690, '13': 720, '14': 270, '15': 210, '16': 150, '17': 360, '18': 390, '19': 60, '20': 90, '21': 90, '22': 120, '23': 150, '24': 90, '25': 480, '26': 180, '27': 60, '28': 150, '29': 90, '30': 150, '31': 270, '32': 60, '33': 210, '34': 120, '35': 390, '36': 120, '37': 60, '38': 690, '39': 90, '40': 90, '41': 60, '42': 90}
Total images across all categories for testing: 12630


In [4]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
])

dataset_train = ImageFolder(
    'Train',
    transform = train_transforms
)
dataset_test = ImageFolder(
    'Test_organized',
    transform = train_transforms
)

In [5]:
train_data, val_data = random_split(dataset_train, [0.8, 0.2])

https://github.com/poojahira/gtsrb-pytorch

### CNN Model - 1

In [6]:
wandb.init(project="Project CNN_v1", name = 'experiment_v1')

In [7]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(dataset_test, shuffle=True, batch_size = 64, num_workers = 2)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Define feature extractor
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),

            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.3),

            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.4),

            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.5)
            )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 2 * 2, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features=1000, out_features=256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
            )

    def forward(self, x):
        # Pass input through feature extractor and classifier
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the features
        x = self.classifier(x)
        return x


In [9]:
net = Net(num_classes=43)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
summary(net,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           1,792
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         Dropout2d-4           [-1, 64, 16, 16]               0
         MaxPool2d-5             [-1, 64, 8, 8]               0
            Conv2d-6            [-1, 192, 8, 8]         110,784
       BatchNorm2d-7            [-1, 192, 8, 8]             384
              ReLU-8            [-1, 192, 8, 8]               0
         Dropout2d-9            [-1, 192, 8, 8]               0
        MaxPool2d-10            [-1, 192, 4, 4]               0
           Conv2d-11            [-1, 384, 4, 4]         663,936
      BatchNorm2d-12            [-1, 384, 4, 4]             768
             ReLU-13            [-1, 384, 4, 4]               0
        Dropout2d-14            [-1, 38

In [10]:
wandb.config.update({
    "learning_rate": 0.001,
    "epochs": 100,
    "batch_size": 64
})
wandb.watch(net)

[]

In [11]:
class EarlyStopper:
    def __init__(self, patience=1, delta=0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.min_validation_loss = float('inf')
        self.best_model = None

    def early_stop(self, validation_loss, model):
        if validation_loss < self.min_validation_loss - self.delta:
            self.min_validation_loss = validation_loss
            self.counter = 0
            self.best_model = model.state_dict()
        elif validation_loss >= self.min_validation_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                model.load_state_dict(self.best_model)
                return True
        return False


In [12]:
def get_loss(loader, model, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()  # Sum up batch loss

    average_loss = total_loss / len(loader)
    return average_loss


In [13]:
def get_accuracy(dataloader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [14]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
early_stopping = EarlyStopper(patience=5,delta = 0.01)

In [15]:
epochs = 100
model = net
for epoch in range(epochs):
    # Training loop
    model.train()
    for i,(images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i %100 == 0:
          print("Epoch", epoch+ 1, " batch", i+1, ". Training Loss: ", loss.item())
          if enable_wandb:
            wandb.log({"loss": loss})
        wandb.log({"train_loss":loss.item()})

    # Validation loop
    model.eval()
    validation_loss = get_loss(val_loader, model, criterion, device)
    validation_accuracy = get_accuracy(val_loader, model, device)
    wandb.log({"val_loss": validation_loss, "val_accuracy": validation_accuracy})

    print(f'Epoch {epoch+1}: Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.2f}%')

    if early_stopping.early_stop(validation_loss, model):
        print("Early stopping triggered.")
        break
wandb.finish()


Epoch 1  batch 1 . Training Loss:  3.740623712539673
Epoch 1  batch 101 . Training Loss:  3.2844128608703613
Epoch 1  batch 201 . Training Loss:  2.627850294113159
Epoch 1  batch 301 . Training Loss:  2.0726101398468018
Epoch 1  batch 401 . Training Loss:  2.0345070362091064
Epoch 1: Validation Loss: 1.6195, Validation Accuracy: 42.10%
Epoch 2  batch 1 . Training Loss:  1.851232886314392
Epoch 2  batch 101 . Training Loss:  1.8435171842575073
Epoch 2  batch 201 . Training Loss:  1.7504080533981323
Epoch 2  batch 301 . Training Loss:  1.6081430912017822
Epoch 2  batch 401 . Training Loss:  1.6278976202011108
Epoch 2: Validation Loss: 1.2251, Validation Accuracy: 56.70%
Epoch 3  batch 1 . Training Loss:  1.5973659753799438
Epoch 3  batch 101 . Training Loss:  1.3043276071548462
Epoch 3  batch 201 . Training Loss:  1.249068260192871
Epoch 3  batch 301 . Training Loss:  1.0592491626739502
Epoch 3  batch 401 . Training Loss:  0.9640299677848816
Epoch 3: Validation Loss: 0.7157, Validation A

loss,█▆▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▅▄▄▃▃▃▂▂▁▂▂▂▁▂▁▁▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▅▇▇▇█████████████████
val_loss,█▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.08109
train_loss,0.00345
val_accuracy,98.99248
val_loss,0.03599


------------------------------------------------------------------------

### CNN Model - 2

- Decreasing `epochs` from 100 to 25
- Increasing `batch_size` from 64 to 128

In [25]:
wandb.init(project="Project CNN_v1", name = 'experiment_v2')

In [26]:
wandb.config.update({
    "learning_rate": 0.001,
    "epochs": 25,
    "batch_size": 128
})
wandb.watch(net)

[]

In [27]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(dataset_test, shuffle=True, batch_size = 128, num_workers = 2)

In [28]:
net1 = Net(num_classes=43)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net1.to(device)

Net(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.2, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Dropout2d(p=0.3, inplace=False)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): Dropout2d(p=0.4, inplace=False)
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): BatchN

In [29]:
optimizer = optim.Adam(net1.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
early_stopping = EarlyStopper(patience=5,delta = 0.01)

In [30]:
epochs = 25
model = net1
for epoch in range(epochs):
    # Training loop
    model.train()
    for i,(images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i %100 == 0:
          print("Epoch", epoch+ 1, " batch", i+1, ". Training Loss: ", loss.item())
          if enable_wandb:
            wandb.log({"loss": loss})
        wandb.log({"train_loss":loss.item()})

    # Validation loop
    model.eval()
    validation_loss = get_loss(val_loader, model, criterion, device)
    validation_accuracy = get_accuracy(val_loader, model, device)
    wandb.log({"val_loss": validation_loss, "val_accuracy": validation_accuracy})

    print(f'Epoch {epoch+1}: Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.2f}%')

    if early_stopping.early_stop(validation_loss, model):
        print("Early stopping triggered.")
        break
wandb.finish()


Epoch 1  batch 1 . Training Loss:  3.854710817337036
Epoch 1  batch 101 . Training Loss:  2.9450109004974365
Epoch 1  batch 201 . Training Loss:  2.075486183166504
Epoch 1: Validation Loss: 1.8634, Validation Accuracy: 37.39%
Epoch 2  batch 1 . Training Loss:  2.2888221740722656
Epoch 2  batch 101 . Training Loss:  1.9419071674346924
Epoch 2  batch 201 . Training Loss:  1.423435926437378
Epoch 2: Validation Loss: 1.1884, Validation Accuracy: 57.85%
Epoch 3  batch 1 . Training Loss:  1.5208704471588135
Epoch 3  batch 101 . Training Loss:  1.1274008750915527
Epoch 3  batch 201 . Training Loss:  0.9119271039962769
Epoch 3: Validation Loss: 0.6026, Validation Accuracy: 78.66%
Epoch 4  batch 1 . Training Loss:  1.120024561882019
Epoch 4  batch 101 . Training Loss:  0.7450882196426392
Epoch 4  batch 201 . Training Loss:  0.6549537777900696
Epoch 4: Validation Loss: 0.3837, Validation Accuracy: 87.20%
Epoch 5  batch 1 . Training Loss:  0.695164680480957
Epoch 5  batch 101 . Training Loss:  0.

loss,█▆▅▄▃▃▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▅▄▄▃▃▂▂▂▂▁▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▆▇▇▇██████████████████
val_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.04553
train_loss,0.00129
val_accuracy,99.069
val_loss,0.03595


---------------------------------

### CNN Model - 3

- Balance the training data
- Still using `batch_size` of 64 since it gives higher accuracy

In [36]:
wandb.init(project="Project CNN_v1", name = 'experiment_v3')

In [37]:
wandb.config.update({
    "learning_rate": 0.001,
    "epochs": 25,
    "batch_size": 64
})
wandb.watch(net)

[]

In [38]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(dataset_test, shuffle=True, batch_size = 64, num_workers = 2)

In [39]:
net2 = Net(num_classes=43)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net2.to(device)

Net(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.2, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Dropout2d(p=0.3, inplace=False)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): Dropout2d(p=0.4, inplace=False)
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): BatchN

In [40]:
weights = [total_train/sorted_train[str(i).zfill(2)] if str(i).zfill(2) in sorted_train else 1 for i in range(43)]
class_weights = torch.FloatTensor(weights).cuda()

In [41]:
optimizer = optim.Adam(net2.parameters(), lr=0.001)
early_stopping = EarlyStopper(patience=5,delta = 0.01)
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [42]:
epochs = 25
model = net1
for epoch in range(epochs):
    # Training loop
    model.train()
    for i,(images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i %100 == 0:
          print("Epoch", epoch+ 1, " batch", i+1, ". Training Loss: ", loss.item())
          if enable_wandb:
            wandb.log({"loss": loss})
        wandb.log({"train_loss":loss.item()})

    # Validation loop
    model.eval()
    validation_loss = get_loss(val_loader, model, criterion, device)
    validation_accuracy = get_accuracy(val_loader, model, device)
    wandb.log({"val_loss": validation_loss, "val_accuracy": validation_accuracy})

    print(f'Epoch {epoch+1}: Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.2f}%')

    if early_stopping.early_stop(validation_loss, model):
        print("Early stopping triggered.")
        break
wandb.finish()


Epoch 1  batch 1 . Training Loss:  0.05538102611899376
Epoch 1  batch 101 . Training Loss:  0.15883882343769073
Epoch 1  batch 201 . Training Loss:  0.011291002854704857
Epoch 1  batch 301 . Training Loss:  0.14571800827980042
Epoch 1  batch 401 . Training Loss:  0.09036130458116531
Epoch 1: Validation Loss: 0.0486, Validation Accuracy: 99.17%
Epoch 2  batch 1 . Training Loss:  0.08589104562997818
Epoch 2  batch 101 . Training Loss:  0.044697053730487823
Epoch 2  batch 201 . Training Loss:  0.09368452429771423
Epoch 2  batch 301 . Training Loss:  0.02099890261888504
Epoch 2  batch 401 . Training Loss:  0.01709175668656826
Epoch 2: Validation Loss: 0.0508, Validation Accuracy: 99.11%
Epoch 3  batch 1 . Training Loss:  0.042652010917663574
Epoch 3  batch 101 . Training Loss:  0.05108672007918358
Epoch 3  batch 201 . Training Loss:  0.0346488282084465
Epoch 3  batch 301 . Training Loss:  0.40025195479393005
Epoch 3  batch 401 . Training Loss:  0.19807152450084686
Epoch 3: Validation Loss:

loss,▂▄▁▃▂▂▂▂▁▁▂▂▁█▄▃▄▁▅▂▂▂▃▂▂▃▁▂▂▂
train_loss,▁▃▃▁██▄▃▂▂▂▁▁▃▂▄▂▂▆▂▁▁▂▂▂▂▁▄▃▃▃▁▂▁▃▁▃▁▁▅
val_accuracy,▇▁█▆▅▅
val_loss,▃█▁▂█▄
loss,0.07442
train_loss,6e-05
val_accuracy,99.14552
val_loss,0.04917
